# Observation blocks

Database name is 'papahana' collection name is 'ob_block'. Create collection assumes to be assessed through port 27017.

In [1]:
from generate_demo_database import *
import sys
sys.path.append('papahana_flask_server_demo/swagger_server/controllers')
from controller_helper import *
dbName = 'papahana'
collName = 'ob_block'
nOb = 100
coll = create_collection(dbName, collName, port=27017)

## OB queries
The following queries can be called by the API, returning dictionaries as JSON objects. The first objects of each query are printed out. You are encouraged to alter the search queries yourself. 

In [5]:
sig = {
		"pi" : "Buster Bluth",
		"semesters" : ["2020A"],
		"program" : 2,
		"observers" : [
			"Herbert Love",
			"Oscar Bluth",
			"Tony Wonder",
			"Mort Meyers"
		],
	}

queries = (
    coll.find(),
    find_by_pi(sig["pi"], coll),
    find_by_observer(sig['observers'][0], coll),
    find_by_observer_semester(sig['observers'][0], sig['semesters'][0], coll)
)

for cursor in queries:
    obExample = cursor.next()
    pprint.pprint(obExample['signature'], depth=2)

{'comment': 'Here’s some money. Go see a star war.',
 'observers': ['Annyong Bluth', 'Lottie Dottie Da'],
 'pi': 'Lucille Bluth',
 'program': 6,
 'semesters': ['2028A', '2023A', '2004B']}
{'comment': 'I hear the jury’s still out on science.',
 'observers': ['Carl Weathers', 'Argyle Austero'],
 'pi': 'Buster Bluth',
 'program': 10,
 'semesters': ['2026B', '2010B', '2021A']}
{'comment': 'I don’t understand the question and I won’t respond to it.',
 'observers': ['Herbert Love', 'Cindi Lightballoon', 'Kitty Sanchez'],
 'pi': 'Michael Bluth',
 'program': 4,
 'semesters': ['2028A', '2023A', '2020A']}
{'comment': 'I don’t understand the question and I won’t respond to it.',
 'observers': ['Herbert Love', 'Cindi Lightballoon', 'Kitty Sanchez'],
 'pi': 'Michael Bluth',
 'program': 4,
 'semesters': ['2028A', '2023A', '2020A']}


In [6]:
aggregates = (
get_distinct_semesters(sig['observers'][0], coll),  # /semIds/{semester} (get) 
get_semesters_by_pi(sig['pi'], coll),  # /semIds (get) # Retrieves all the programs (semesters) associated with a PI
get_ob_by_semester(sig['semesters'][0], coll), #  /semIds/{semester} (get) retrieve the programs for the given semester
get_ob_by_semester_observer(sig['semesters'][0], sig['observers'][0], coll)  # /semIds/{semId} (get)  Retrieves the specified program
)

for cursor in aggregates:
    obExample = cursor.next()
    pprint.pprint(obExample, depth=2)

{'_id': {'semester': None}}
{'_id': '16', 'signature': {}}


StopIteration: 

In [11]:
ob = list(get_ob_by_id('17', coll))[0]
ob['_id'] = str(101)
delete_observation_block(str(101), coll)
insert_observation_block(ob, coll)
newValues = {"$set": { "priority": 100 } }
update_observation_block(str(101), newValues, coll)
ob = list(get_ob_by_id('101', coll))
print(ob)

[{'_id': '101', 'signature': {'pi': 'Gob Bluth', 'semester': '2002A', 'program': 6, 'observers': ['Stan Sitwell', 'Steve Holt', 'Argyle Austero', 'Stefan Gentles'], 'group': 1}, 'version': '0.1', 'acquisition': {'name': 'KCWI_ifu_acq_direct', 'version': '0.1', 'script': 'KCWI_ifu_acq_direct', 'guider_po': 'IFU', 'guider_gs_ra': 5.993199268191816, 'guider_gs_dec': 40.0944994434565, 'guider_gs_mode': 'User'}, 'associations': ['rnynw', 'ajkjy', 'zdkyu', 'tdpgq'], 'priority': 100, 'status': 'broken', 'comment': 'I hear the jury’s still out on science.'}]


# Groups

In [14]:
groups = ['Army', 'The Alliance of Magicians', 'Tantamount Studios', 'Orange County Prison', 'Milford School', 'Dr. Fünke\'s 100% Natural Good-Time Family-Band Solution']
randGroupName = lambda: random.choice(groups)
randOBIds = lambda x=5: [int(x) for x in list(np.random.choice( range(0,nOb+1), size=random.randint(0, x), replace=False))]
def generate_group(_id=None):
    schema = {
        "semester": randSemester(),
        "name": randGroupName(),
        "comment": randComment(),
        "observation_blocks": randOBIds()
    }
    if _id:
        schema['_id'] = _id
    return schema

In [15]:
dbName = 'papahana'
collName = 'groups'
remote=True # run on remote server (n)
coll = create_collection(dbName, collName, port=27017, remote=False)
coll.drop()
nGroups = 20
for idx in range(nGroups):
    doc = generate_group(str(idx))
    result = coll.insert_one(doc)
    assert result.inserted_id == str(idx), 'check that idx was sed properly'